In [1]:
import xarray as xr
import geopandas as gpd
import regionmask
import numpy as np
import copernicusmarine as cm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#monthly bottom temp data from copernicus
#ds_gl = cm.open_dataset(dataset_id="cmems_mod_glo_phy_my_0.083deg_P1M-m")

In [ ]:
#bt_gl = ds_gl.bottomT.sel(latitude = slice(30, 48), longitude = slice(-80, -60))

In [ ]:
#bt_gl.to_netcdf('bt_glorys_M.nc')

In [14]:
opendap_url = "http://psl.noaa.gov/thredds/dodsC/Projects/CEFI/regional_mom6/northwest_atlantic/hist_run/regrid/ocean_monthly.199301-201912.tob.nc"
bt_m6 = xr.open_dataset(opendap_url)

In [2]:
bt_gl = xr.open_dataset('bt_glorys_M.nc')

In [ ]:
gl_regrid = np.empty([324, 774, 844])
#gl_lons = np.concatenate((np.array(bt_glorys.longitude[:2161]+360), np.array(bt_glorys.longitude[2161:])))
LAT, LON = np.meshgrid(bt_mom6.lat.values, bt_mom6.lon.values-360)
for i in range(324):
    interp_to_mom6 = RegularGridInterpolator((bt_glorys.latitude.values, bt_glorys.longitude.values), bt_glorys.bottomT[i,:,:].values, bounds_error = False, fill_value = np.nan)
    gl_regrid[i,:,:] = interp_to_mom6((LAT, LON))

In [3]:
strata = gpd.read_file('EcoMon_strata/EcoMon_Strata.shp')

In [4]:
# Get the region of interest
region = regionmask.from_geopandas(strata)

In [5]:
# Create the mask
mask = region.mask(bt_gl.longitude, bt_gl.latitude)

In [11]:
bt_gl_strata = xr.Dataset(data_vars = {'01': (['time','latitude','longitude'], bt_gl.bottomT.where(mask == region.numbers[0]).data)},
                          coords = {'time': bt_gl.time,
                                    'latitude': bt_gl.latitude,
                                    'longitude': bt_gl.longitude})

In [12]:
for i in np.arange(1, len(region.numbers)):
    masked_ds = bt_gl.where(mask == region.numbers[i])
    strata_bt_gl_n = xr.Dataset(data_vars = {strata.STRATA[i]: (['time','latitude','longitude'], bt_gl.bottomT.where(mask == region.numbers[i]).data)},
                          coords = {'time': bt_gl.time,
                                    'latitude': bt_gl.latitude,
                                    'longitude': bt_gl.longitude})
    bt_gl_strata = xr.merge([bt_gl_strata, strata_bt_gl_n])

In [13]:
bt_gl_strata

<xarray.Dataset>
Dimensions:    (time: 342, latitude: 217, longitude: 241)
Coordinates:
  * time       (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2021-06-01
  * latitude   (latitude) float32 30.0 30.08 30.17 30.25 ... 47.83 47.92 48.0
  * longitude  (longitude) float32 -80.0 -79.92 -79.83 ... -60.17 -60.08 -60.0
Data variables: (12/47)
    01         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    02         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    03         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    04         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    05         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    06         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    ...         ...
    42         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    43         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    44         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    45         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    46         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    47         (time, latitude, longitude) float32 nan nan nan ... nan nan nan